In [5]:
%matplotlib inline
import netCDF4
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import datetime

import os

In [6]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [7]:
def create_netCDF_Haukeli(year,month,day,stn_lat,stn_lon,var_name):
    thredds  = 'http://thredds.met.no/thredds/dodsC/meps25epsarchive'
    time = '00'
    fn = netCDF4.Dataset('%s/%s/%s/%s/meps_full_2_5km_%s%s%sT%sZ.nc' %(thredds,year,month,day,year,month,day,time),'r')


## Latitudes
## [y = 949][x = 739]
    latitude = fn.variables['latitude']

## Longitudes 
## [y = 949][x = 739]
    longitude = fn.variables['longitude']

### From the with Vertical Levels ( height0 ) : 0.0 m

## 1. air_temperature_0m = Surface temperature (T0M)
## [time = 56][height0 = 1][ensemble_member = 10][y = 949][x = 739]
#var_name = 'air_temperature_0m'
    temp_sfc = fn.variables[var_name]    # [K]


# Stations latitude longitude, the nearest gridpoint we are looking for
#stn_lat = 59.8
#stn_lon = 7.2

# Now find the absolute value of the difference between the  station's lat/lon with every point in the grid. 
# This tells us how close a point is to the particular latitude and longitude.

    abslat = np.abs(latitude[:,:]-stn_lat)
    abslon= np.abs(longitude[:,:]-stn_lon)

# Now we need to combine these two results. We will use numpy.maximum, which takes two arrays and finds the local 
# maximum.
    c = np.maximum(abslon, abslat)

# If you don't like flattened arrays, you can also get the row/column index like this
    y, x = np.where(c == np.min(c))

### fill values of missing data
    mask = temp_sfc[:,:,:,y[0],x[0]].mask
    fill_value = temp_sfc[:,:,:,y[0],x[0]].fill_value

    marr = np.ma.array(temp_sfc[:,:,:,y[0],x[0]], mask = mask, fill_value = fill_value)
    dt = marr.filled().dtype


    
    
### write netCDF file 
    f_sfc = netCDF4.Dataset('../dataMEPS/%s/%s%s%s.nc' %(var_name,year,month,day), 'w')
    f_sfc.createDimension('time', 67)            # [time = 67]
    f_sfc.createDimension('height0', 1)          # [height0 = 1]
    f_sfc.createDimension('hybrid', 65)          # [hybrid = 65]
    f_sfc.createDimension('height_above_msl', 1) # [height_above_msl = 1]
    f_sfc.createDimension('ensemble_member', 10)  # [ensemble_member = 10]
    f_sfc.createDimension('y', 1)                # [y = 1]
    f_sfc.createDimension('x', 1)                # [x = 1]

    grid_temp_sfc = f_sfc.createVariable('%s' %(var_name), dt, ('time','height0','ensemble_member','y','x'),fill_value = fill_value,zlib=True)



    grid_temp_sfc[:] = marr.filled()

    f_sfc.close()

In [8]:
# Stations latitude longitude, the nearest gridpoint we are looking for
stn_lat = 59.8
stn_lon = 7.2

year = '2016'
month = '12'

# Variable
var_name = [ \
    ### From the with Vertical Levels ( height0 ) : 0.0 m
            'air_temperature_0m',                           # [K]
            'graupelfall_amount',                           # [kg/m^2]
            'liquid_water_content_of_surface_snow',         # [kg/m^2]
            'precipitation_amount_acc',                     # [kg/m^2] 
            'rainfall_amount',                              # [kg/m^2]
            'snowfall_amount',                              # [kg/m^2]
    ### with Vertical Levels ( height_above_msl ) : 0.0 m
            'air_pressure_at_sea_level',                    # [Pa]  
    ### From with Vertical Levels ( hybrid )
 #           'air_temperature_ml',                           # [K]                   
  #          'atmosphere_cloud_condensed_water_content_ml',  # [kg/m^2]
   #         'atmosphere_cloud_ice_content_ml',              # [kg/m^2]
    #        'graupelfall_amount_ml',                        # [kg/m^2]
     #       'pressure_departure',                           # [Pa]
      #      'rainfall_amount_ml',                           # [kg/m^2]
       #     'snowfall_amount_ml',                           # [kg/m^2]
        #    'specific_humidity_ml'                          # [kg/kg]
           ]                         


    



for name in var_name:
    createFolder('../dataMEPS/%s/' %(name))
    # Date
    for day in range(17,31):
        create_netCDF_Haukeli(year,month,day,stn_lat,stn_lon,name)

ValueError: cannot reshape array of size 600 into shape (67,1,10,1,1)